In [ ]:
#import library
import numpy as np
import pandas as pd

In [ ]:
# Importing and reading dataset
ds=pd.read_csv("New_cleaned_data.csv")
ds

In [ ]:
# Remove unnamed column

ds = ds.loc[:, ~ds.columns.str.contains('^Unnamed')]                
ds

In [4]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651 entries, 0 to 650
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        651 non-null    object 
 1   company     651 non-null    object 
 2   year        651 non-null    int64  
 3   Price       651 non-null    float64
 4   kms_driven  651 non-null    int64  
 5   fuel_type   651 non-null    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 30.6+ KB


In [ ]:
# Spliting data into Independent and dependent variables

x=ds[['company','name','year','kms_driven','fuel_type']]
y=ds[['Price']]

In [ ]:
# Cheack lenth

print(len(ds['company'].unique()))
print(len(ds['name'].unique()))
print(len(ds['fuel_type'].unique()))

In [ ]:
# Implement OneHotEncoder

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer   # dataframe create then categorical to numeric and vice_versa
ohe=OneHotEncoder()
ohe.fit(x[["company","name","fuel_type"]])

In [ ]:
# To categories check

ohe.categories_

In [ ]:
ct=make_column_transformer((OneHotEncoder(handle_unknown='ignore',categories=ohe.categories_),["company","name","fuel_type"]),remainder='passthrough',force_int_remainder_cols=False,sparse_threshold=0)
ct                          

In [ ]:
# making pipeline

from sklearn.linear_model import LinearRegression
reg=LinearRegression()


In [ ]:
from sklearn.pipeline import make_pipeline
pipe=make_pipeline(ct,reg)
pipe

In [ ]:
# Spliting dataset into training and testing

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
scores=[]
for i in range(0,101):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.10,random_state=i)
    pipe.fit(x_train,y_train)
    result=pipe.predict(x_test)
    score=r2_score(y_test,result)
    scores.append(score)

In [ ]:
# Finding best value

bestindex=np.argmax(scores)
scores[bestindex]


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.10,random_state=bestindex)
pipe.fit(x_train,y_train)

In [ ]:
# Predict for user_input

company=input("Enter company name : ")
name=input("Enter car name : ")
year=int(input("Enter year : "))
kms_driven=int(input("Enter kms_driven : "))
fuel_type=input("Enter fuel type : ")
columns=["company","name","year","kms_driven","fuel_type"]

myinput=pd.DataFrame(columns=columns,data=[[company,name,year,kms_driven,fuel_type]])
result=pipe.predict(myinput)

print("You should buy it for ~ price :",abs(round(result[0,0])))

In [ ]:
import pickle as pkl

In [ ]:
pkl.dump(pipe,open("CarProject.pkl","wb"))